In [ ]:
import io
import base64
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
#from loguru import logger
import numpy as np
import time
import pickle
import h5py
#from skimage.transform import downscale_local_mean,resize
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans,DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances,pairwise_distances,pairwise_distances_argmin_min
import umap
import scipy
import time

from dash import Dash, dcc, html, Input, Output, no_update, callback, jupyter_dash
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html
import dash_vtk
from dash_vtk.utils import to_volume_state,to_mesh_state
import vtk
import dash_bootstrap_components as dbc
import colorcet as cc

from PIL import Image

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np

import itk
from itkwidgets import view
import SimpleITK as sitk

from utilities import plot_all_slices_notzero

In [ ]:
st=time.time()
with h5py.File("dataset_humerus_labelmap_res.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
humerus_dataset.shape

In [ ]:
with open("processing/humerus_res_features_250ep.pkl","rb") as f:
    features = pickle.load(f)

In [ ]:
with open("processing/humerus_res_flip_features_250ep.pkl","rb") as f:
    features = pickle.load(f)

In [ ]:
features.shape

## Calcolo array dataset-CT

In [ ]:
with open("humerus_preprocessing/arrays/pos_array_hor_all.pkl","rb") as f1:
    pos_array_hor_all = pickle.load(f1)

In [ ]:
# Devo far corrispondere ad ogni elemento del dataset la sua CT originale
dataset_el_to_ct = [] # In corrispondenza dell'indice l'elemento conterrà la CT corrispondente
for i,el in enumerate(pos_array_hor_all,start=1):
    if len(el) != 0:
        dataset_el_to_ct.append(i)
    if len(el) > 2:
        dataset_el_to_ct.append(i)

In [ ]:
dataset_el_to_ct.index(859) # 859 è la CT e 789 l'indice della CT nel dataset

## Confronto dataset-ct (tesi)

In [ ]:
dataset_el = 0
dataset_img = humerus_dataset[dataset_el]

In [ ]:
left_humerus_label = 69
right_humerus_label = 70
left_scapula_label = 71
right_scapula_label = 72
shoulder_seg_data = np.asarray(nib.load(f"processing/{dataset_el_to_ct[dataset_el]}/shoulder_seg.nii").dataobj)
shoulder_seg_humerus_data = np.where((shoulder_seg_data == left_humerus_label) | (shoulder_seg_data == right_humerus_label),1,0).astype(np.uint8)

In [ ]:
view(dataset_img*255)

In [ ]:
view(shoulder_seg_humerus_data*255)

In [ ]:
view(shoulder_seg_data*255)

## Clustering e visualizzazione

### Metriche cluster

In [ ]:
# Stampo l'inerzia (distanza within cluster) per vari numeri di cluster per vedere qual è il numero di cluster ottimale (gomito)
distortions = []

for k in range(1,30):
    kmeanModel = KMeans(n_clusters=k,n_init="auto")
    labels = kmeanModel.fit_predict(features)
    distortions.append(kmeanModel.inertia_)

In [ ]:
sns.lineplot(x=range(1,30),y=distortions,marker="o")
plt.title("Inertia")

### Clustering (10 cluster)

In [ ]:
#number_clusters = 10
number_clusters = 15
kmeans = KMeans(n_clusters=number_clusters,n_init="auto",random_state=42)
labels = kmeans.fit_predict(features)

#### Metriche dopo clustering

In [ ]:
centroids = kmeans.cluster_centers_
unique_labels,cardinalities = np.unique(labels,return_counts=True)

In [ ]:
# Distanze dei centroidi (intra cluster)
centroid_distances = pairwise_distances(centroids,metric="euclidean")
plt.imshow(centroid_distances)
plt.title("Matrice delle distanze dei centroidi negli Omeri")
plt.xticks(unique_labels)
plt.yticks(unique_labels)
plt.colorbar()

In [ ]:
# Cardinalità cluster (inter cluster)
plt.bar(x=unique_labels,height=cardinalities)
plt.title("Cardinalità dei Cluster negli Omeri")
plt.xticks(unique_labels)
plt.xlabel("Cluster")
plt.ylabel("Cardinalità")

In [ ]:
# Magnitudine(media della somma delle distanze di ogni esempio dal suo centroide) (inter cluster)
dists = pairwise_distances(features,centroids)
dist_features_centroids = {el:[] for el in unique_labels}
for dist,lab in zip(dists,labels):
    dist_features_centroids[lab].append(dist[lab])
mean_dists = [np.mean(dist_features_centroids[i]) for i in unique_labels]                                        

In [ ]:
plt.bar(x=unique_labels,height=mean_dists)
plt.title("Distanza Media degli Elementi dal Centroide negli Omeri")
plt.xticks(unique_labels)
plt.xlabel("Cluster")
plt.ylabel("Distanza media dal centroide")

#### Visualizzazione

In [ ]:
st = time.time()
features_tsne = TSNE(random_state=42).fit_transform(features) # Per la visualizzazione in 2D
print(f"Tempo dimensionality reduction TSNE: {time.time()-st:.2f} sec")
features_tsne.shape

In [ ]:
plt.figure(figsize=(15,10))
plt.title("TSNE Omeri",fontsize=20)
sns.scatterplot(
    x = features_tsne[:,0],
    y = features_tsne[:,1],
)

In [ ]:
#plt.figure(figsize=(15,10))
fig,ax = plt.subplots(figsize=(15,10))
plt.title("Clustering TSNE Omeri",fontsize=24)
sns.scatterplot(
    x = features_tsne[:,0],
    y = features_tsne[:,1],
    hue = labels,
    palette = sns.color_palette(cc.glasbey_bw, n_colors=number_clusters),
    ax=ax
)
# Cluster centers
#sns.scatterplot(x=centers[:,0],y=centers[:,1],marker="x",ax = ax,s=150,color="black")

In [ ]:
# UMAP
st = time.time()
features_umap = umap.UMAP(n_neighbors=30,random_state=42).fit_transform(features)
print(f"Tempo dimensionality reduction UMAP: {time.time()-st:.2f} sec")
features_umap.shape

In [ ]:
plt.figure(figsize=(15,10))
plt.title("UMAP Omeri",fontsize=20)
sns.scatterplot(
    x = features_umap[:,0],
    y = features_umap[:,1],
)

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Clustering UMAP Omeri",fontsize=24)
sns.scatterplot(
    x = features_umap[:,0],
    y = features_umap[:,1],
    hue= labels,
    palette = sns.color_palette(cc.glasbey_bw, n_colors=number_clusters)
)

### Visualizzazione per volume

Per ogni CT colore in base al suo volume (somma voxel)

In [ ]:
#Calcolo dei volumi di ogni CT e poi normalizzo sul volume più grande
#Ricordare che ogni voxel ha dimensione 0.5mm in tutte e tre le direzioni
vol = np.sum(humerus_dataset,axis=(1,2,3)) # Serve il dataset
maxvol = np.max(vol)
rel_vols = vol / maxvol
rel_vols.shape

In [ ]:
with open("processing/humerus_volumes.pkl","wb") as f:
    pickle.dump(rel_vols,f)

In [ ]:
with open("processing/humerus_volumes.pkl","rb") as f:
    rel_vols = pickle.load(f)
rel_vols.shape

In [ ]:
fig,ax = plt.subplots(figsize=(15,10))
plt.title("UMAP Volumes",fontsize=24)
pts = ax.scatter(
    x=features_umap[:,0],
    y=features_umap[:,1],
    c=rel_vols,
    s=20,
    cmap="plasma"
)
fig.colorbar(pts)
plt.show()

## Coregistrazione rappresentanti cluster 

In [ ]:
# Prendo i 4 punti vicini ad ogni centroide. Sono in ordine di distanza, il primo è il più vicino
neighbors = NearestNeighbors(n_neighbors=4).fit(features)
distances,indexes = neighbors.kneighbors(kmeans.cluster_centers_)

In [ ]:
distances,indexes

In [ ]:
indexes[0][0],indexes[1][0] # Indice del punto più vicino al centroide del cluster 0

In [ ]:
dataset_el_to_ct[indexes[0][0]],dataset_el_to_ct[indexes[1][0]] 
# Il numero di CT corrispondente all'indice del pto più vicino al centroide

### Con immagini itk

In [ ]:
ct=844

In [ ]:
# Segmento l'omero dal nifti originale per poi fare coregistrazione
original_shoulder = nib.load(f"processing/{ct}/shoulder_nifti.nii")
segmented_shoulder = nib.load(f"processing/{ct}/shoulder_seg.nii")

original_shoulder_data = np.asarray(original_shoulder.dataobj)
segmented_shoulder_data = np.asarray(segmented_shoulder.dataobj)

hum_labels = [69,70]
original_shoulder_data_segmented_humerus = np.where((segmented_shoulder_data == hum_labels[0]) | (segmented_shoulder_data==hum_labels[1]),original_shoulder_data,0)

original_shoulder_segmented_humerus = nib.Nifti1Image(original_shoulder_data_segmented_humerus,original_shoulder.affine)
nib.save(original_shoulder_segmented_humerus,f"processing/{ct}/shoulder_humerus_nifti_segm.nii")

In [ ]:
fixed_im = itk.imread("processing/1025/shoulder_humerus_nifti_segm.nii",itk.F)
moving_im = itk.imread("processing/844/shoulder_humerus_nifti_segm.nii",itk.F)

In [ ]:
# Importo i parametri in base al compito che devo fare
parameter_object = itk.ParameterObject.New()
parameter_object.AddParameterFile('registration-params/Par0004.rigid.txt')

In [ ]:
st = time.time()
registered_image, params = itk.elastix_registration_method(fixed_im, moving_im) # Ha la shape della fixed image(primo arg)
print(f"Tempo: {time.time()-st:.2f} sec")

In [ ]:
registered_image.shape,fixed_im.shape,moving_im.shape

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,10))
sl = 250
ax[0].imshow(registered_image[sl])
ax[0].set_title("Registered")
ax[1].imshow(fixed_im[sl])
ax[1].set_title("Fixed")
ax[2].imshow(fixed_im[sl]-registered_image[sl])
ax[2].set_title("Subtraction")

In [ ]:
st = time.time()
registered_image, params = itk.elastix_registration_method(moving_im, fixed_im)
print(f"Tempo: {time.time()-st:.2f} sec")

In [ ]:
registered_image.shape,fixed_im.shape,moving_im.shape

In [ ]:
np.max(registered_image_norm),np.max(moving_image_norm)

In [ ]:
registered_image_norm = (registered_image - np.min(registered_image))/(np.max(registered_image)-np.min(registered_image))
moving_image_norm = (moving_im - np.min(moving_im))/(np.max(moving_im)-np.min(moving_im))

In [ ]:
sub_img = itk.SubtractImageFilter(moving_im,registered_image)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,10))

sl = 200
ax[0].imshow(registered_image[sl])
ax[0].set_title("Registered")
ax[1].imshow(moving_im[sl])
ax[1].set_title("Fixed")
ax[2].imshow(sub_img[sl])
ax[2].set_title("Subtraction")

### Con numpy di immagini segmentate

In [ ]:
# Segmento l'omero dal nifti originale per poi fare coregistrazione
original_shoulder = nib.load(f"processing/1025/shoulder_nifti.nii")
segmented_shoulder = nib.load(f"processing/1025/shoulder_seg.nii")

original_shoulder_data = np.asarray(original_shoulder.dataobj)
segmented_shoulder_data = np.asarray(segmented_shoulder.dataobj)

hum_labels = [69,70]
fixed_im_np = np.where((segmented_shoulder_data == hum_labels[0]) | (segmented_shoulder_data==hum_labels[1]),original_shoulder_data,0)

# Segmento l'omero dal nifti originale per poi fare coregistrazione
original_shoulder = nib.load(f"processing/844/shoulder_nifti.nii")
segmented_shoulder = nib.load(f"processing/844/shoulder_seg.nii")

original_shoulder_data = np.asarray(original_shoulder.dataobj)
segmented_shoulder_data = np.asarray(segmented_shoulder.dataobj)

hum_labels = [69,70]
moving_im_np = np.where((segmented_shoulder_data == hum_labels[0]) | (segmented_shoulder_data==hum_labels[1]),original_shoulder_data,0)

In [ ]:
registered_image_np, params_np = itk.elastix_registration_method(fixed_im_np, moving_im_np)

In [ ]:
plt.imshow(registered_image_np[250])

### Con numpy labelmap

Default parameters maps:
* translation
* rigid: solo traslazioni e rotazioni
* affine: traslazioni, rotazioni, rescaling e shearing (cambiare assi x e y)
* bspline
* spline
* groupwise

In [ ]:
# Consigli trovati sul manuale
parameter_object = itk.ParameterObject.New()
parameter_map_custom = parameter_object.GetDefaultParameterMap('affine')
# parameter_map_custom['Transform'] = ['SimilarityTransform'] # Solo traslaz, rotaz e scaling isotropico
parameter_map_custom['Metric'] = ['AdvancedKappaStatistic'] # Specifico per binary images
parameter_map_custom['FinalBSplineInterpolationOrder'] = ['0'] # Per produrre una binary image
parameter_object.AddParameterMap(parameter_map_custom)

In [ ]:
indexes # I 4 punti più vicini ai centroidi per ognuno dei 15 cluster

In [ ]:
dataset_el_to_ct[indexes[0][0]] # Per sapere la CT corrispondente al punto

In [ ]:
view(humerus_dataset[indexes[0][0]]*255) # Visualizzazione 3D

In [ ]:
st = time.time()
fixed_im = humerus_dataset[indexes[0][0]]
moving_im = humerus_dataset[indexes[1][0]]
registered_image_labelmap, params_labelmap = itk.elastix_registration_method(fixed_im,moving_im,parameter_object=parameter_object)
print(f"Tempo: {time.time()-st:.2f} sec")

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
# La registered dovrebbe essere simile alla fixed
sl = 250
ax[0].imshow(registered_image_labelmap[:,:,sl])
ax[0].set_title("Registered")
ax[1].imshow(fixed_im[:,:,sl])
ax[1].set_title("Fixed")
ax[2].imshow(moving_im[:,:,sl])
ax[2].set_title("Moving");

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,10))
ax[0,0].imshow(fixed_im[:,:,sl]-registered_image_labelmap[:,:,sl])
ax[0,0].set_title("Fixed - Registered") # è inteso come una sottrazione
ax[0,1].imshow(moving_im[:,:,sl]-registered_image_labelmap[:,:,sl])
ax[0,1].set_title("Moving - Registered");

ax[1,0].imshow(registered_image_labelmap[:,:,sl]-fixed_im[:,:,sl])
ax[1,0].set_title("Registered - Fixed")
ax[1,1].imshow(registered_image_labelmap[:,:,sl]-moving_im[:,:,sl])
ax[1,1].set_title("Registered - Moving");

In [ ]:
# Visualizzazione 3D 
# view(fixed_im-registered_image_labelmap,label_image=fixed_im)
view(registered_image_labelmap-fixed_im)

In [ ]:
view(fixed_im-registered_image_labelmap)

In [ ]:
dataset_el_to_ct[indexes[0][0]],dataset_el_to_ct[indexes[1][0]]

# Dash APP

In [ ]:
# Parametri da cambiare per cambiare i plot
use_tsne = False # True uso TSNE, False uso UMAP

In [ ]:
# Per produrre l'immagine 
def np_image_to_base64(humerus_dataset,slice_x,slice_y,slice_z):
    im_x = Image.fromarray(np.pad(humerus_dataset[slice_x,:,:],((22,21),(0,0)))*255)
    im_y = Image.fromarray(humerus_dataset[:,slice_y,:]*255)
    im_z = Image.fromarray(np.pad(humerus_dataset[:,:,slice_z],((0,0),(152,151)))*255)
    
    img_conc = Image.new("L",(506,246*3))
    img_conc.paste(im_x,(0,0))
    img_conc.paste(im_y,(0,246))
    img_conc.paste(im_z,(0,246*2))
    
    buffer = io.BytesIO()
    img_conc.save(buffer, format="jpeg")
    encoded_image = base64.b64encode(buffer.getvalue()).decode()
    im_url = "data:image/jpeg;base64, " + encoded_image
    return im_url

# DASH app
app = Dash(__name__)

app.layout = html.Div(
    className="container",
    children=[
        html.Div([
            html.P("Modalità di visualizzazione:"),
            dcc.RadioItems(
                id='graph-info', 
                value='Clusters', 
                options=['Clusters', 'Volumes'],
                inline=True,
            )
        ],
        style={"margin-left":"5em"}),
        # dcc.Graph(id="graph-5", figure=fig, clear_on_unhover=True),
        dcc.Graph(id="graph-5", clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip-5", direction='left'),
    ],
)

@app.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Output("graph-5", "figure"),
    Input("graph-5", "hoverData"),
    Input("graph-info", "value"))
def display_hover(hoverData,graph_info):
    # Figura dinamica
    fig = px.scatter(
        x=features_tsne[:,0] if use_tsne else features_umap[:,0],
        y=features_tsne[:,1] if use_tsne else features_umap[:,1],
        color=rel_vols if graph_info=="Volumes" else labels.astype(str), # Trasformo in stringhe per avere discreto
        )

    fig.update_traces(
        hoverinfo="none",
        hovertemplate=None,
    )

    fig.update_layout(
        autosize=False,
        width=1450,
        height=800
    )
    
    if hoverData is None:
        return False, no_update, no_update,fig

    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]
    num = hover_data["pointNumber"]

    #im_matrix = humerus_dataset[num] # Qui va la CT, num è il numero del pallino su cui si passa il puntatore
    #im_url = np_image_to_base64(im_matrix,120,120,250)
    children = [
        html.Div([
            #html.Img(src=im_url,style={"width": "120px", 'display': 'block', 'margin': '0 auto'}),
            html.P("CT " + str(dataset_el_to_ct[num]), style={'font-weight': 'bold'}), # Occhio che Num è il numero nel dataset non la CT
            html.P(f"Volume: {rel_vols[num]:.3f}")
        ])
    ]

    return True, bbox, children, fig

if __name__ == "__main__":
    app.run(jupyter_mode="external",host="0.0.0.0",port="2323")
    app.title = "Humerus TSNE" if use_tsne else "Humerus UMAP"

# Dash Rendering 3D

## Nearest Neighbors

In [ ]:
# Prendo i 4 punti vicini ad ogni centroide. Sono in ordine di distanza, il primo è il più vicino
neighbors = NearestNeighbors(n_neighbors=4).fit(features)
distances,indexes = neighbors.kneighbors(kmeans.cluster_centers_)

In [ ]:
indexes

In [ ]:
centroid_distances = euclidean_distances(kmeans.cluster_centers_)
print(np.round(centroid_distances,decimals=2))

In [ ]:
# Creo il corrispondente array ma con i numeri delle CT al posto degli indici del dataset
indexes_ct = []
for cluster in indexes:
    cluster_ct = []
    for el in cluster:
        cluster_ct.append(dataset_el_to_ct[el])
    indexes_ct.append(cluster_ct)
indexes_ct = np.asarray(indexes_ct)
indexes_ct

with open("processing/humerus_indexes_ct.pkl","wb") as f:
    pickle.dump(indexes_ct,f)

In [ ]:
with open("processing/humerus_indexes_ct_flip.pkl","wb") as f:
    pickle.dump(indexes_ct,f)

## Salvataggio Immagini in VTK

Effettuando un cropping alla bounding box minima per ridurre al massimo le dimensioni e rendere i caricamenti rapidi

In [ ]:
with open("processing/humerus_indexes_ct.pkl","rb") as f:
    indexes_ct = pickle.load(f)

with open("processing/humerus_indexes_ct_flip.pkl","rb") as f:
    indexes_ct_flip = pickle.load(f)

In [ ]:
indexes_ct_flip

In [ ]:
# Se voglio visualizzare le scapole flippate
with open("processing/humerus_flipped_indexes.pkl","rb") as f:
    flipped_idxs = pickle.load(f)

In [ ]:
def bbox_3D(img):

    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return rmin, rmax, cmin, cmax, zmin, zmax

In [ ]:
for ct in indexes_ct_flip.flatten():
    if dataset_el_to_ct.index(ct) in flipped_idxs:
        print(ct)

In [ ]:
# Salvo tutte le immagini in formato vtk con cropping per risparmiare tempo
for ct in indexes_ct_flip.flatten():
    # ct = dataset_el_to_ct[index]
    original_shoulder = nib.load(f"processing/{ct}/shoulder_nifti_res.nii")
    segmented_shoulder = nib.load(f"processing/{ct}/shoulder_seg_res.nii")

    original_shoulder_data = np.asarray(original_shoulder.dataobj)
    segmented_shoulder_data = np.asarray(segmented_shoulder.dataobj)

    left_humerus_label = 69
    right_humerus_label = 70

    # Segmento solo la scapola dai dati originali applicando la maschera
    original_shoulder_data_segmented = np.where((segmented_shoulder_data == left_humerus_label) | (segmented_shoulder_data == right_humerus_label),original_shoulder_data,0)
    
    # Croppo
    bbox = bbox_3D(original_shoulder_data_segmented)
    original_shoulder_data_segmented_cropped = original_shoulder_data_segmented[bbox[0]:bbox[1],bbox[2]:bbox[3],bbox[4]:bbox[5]]
    if dataset_el_to_ct.index(ct) in flipped_idxs:
        original_shoulder_data_segmented_cropped = np.flip(original_shoulder_data_segmented_cropped,axis=0)
        sitk.WriteImage(sitk.GetImageFromArray(original_shoulder_data_segmented_cropped), f"vtk_images/humerus_{ct}_segm_flip_mirrored.vtk")
        print(f"Omero {ct} salvato")
    

## Dash APP Rendering 3D (su file .py)

In [ ]:
# Immagine di esempio per evitare caricamenti lunghi
try:
    from vtkmodules.vtkImagingCore import vtkRTAnalyticSource
except ImportError:
    from vtk.vtkImagingCore import vtkRTAnalyticSource

# Use VTK to get some data
data_source = vtkRTAnalyticSource()
data_source.Update()  # <= Execute source to produce an output
dataset = data_source.GetOutput()

volume_state = to_volume_state(dataset)

content_cube = dash_vtk.View([ 
    dash_vtk.VolumeRepresentation([
        dash_vtk.VolumeController(),
        dash_vtk.Volume(state=volume_state)
    ],
    ),
])

In [ ]:
def create_col(title,image,wid_cols=4,heig=300):
    """
    Crea un'immagine della griglia con titolo e renering
    """
    column = dbc.Col(
                html.Div([
                    html.H4(children=title),
                    html.Div(
                    style={"width": "100%", "height": f"{heig}px"},
                    children=[image]
                    ),
                ]),
                width=wid_cols
            )
    return column

In [ ]:
def create_row(cluster,use_flipped=True,show_flip=True):
    """
    Ritorna le tre immagini più vicine al centroide del cluster passato come parametro e i rispettivi numeri di CT 
    """
    reader = vtk.vtkDataSetReader() # Devo usare questo reader
    if use_flipped:
        ct_0 = indexes_ct_flip[cluster][0]
        ct_1 = indexes_ct_flip[cluster][1]
        ct_2 = indexes_ct_flip[cluster][2]
    else:
        ct_0 = indexes_ct[cluster][0]
        ct_1 = indexes_ct[cluster][1]
        ct_2 = indexes_ct[cluster][2]
    
    if use_flipped:
        if show_flip and os.path.exists(f"vtk_images/humerus_{ct_0}_segm_flip_mirrored.vtk"):
            reader.SetFileName(f"vtk_images/humerus_{ct_0}_segm_flip_mirrored.vtk")
        else:
            reader.SetFileName(f"vtk_images/humerus_{ct_0}_segm_flip.vtk")
    else:
        reader.SetFileName(f"vtk_images/humerus_{ct_0}_segm.vtk")
    reader.Update()
    volume_state_0 = to_volume_state(reader.GetOutput())

    humerus_0 = dash_vtk.View([
        dash_vtk.VolumeRepresentation(
            children=[
                dash_vtk.VolumeController(),
                dash_vtk.Volume(state=volume_state_0),
            ],
        ),
    ])
    
    if use_flipped:
        if show_flip and os.path.exists(f"vtk_images/humerus_{ct_1}_segm_flip_mirrored.vtk"):
            reader.SetFileName(f"vtk_images/humerus_{ct_1}_segm_flip_mirrored.vtk")
        else:
            reader.SetFileName(f"vtk_images/humerus_{ct_1}_segm_flip.vtk")
    else:
        reader.SetFileName(f"vtk_images/humerus_{ct_1}_segm.vtk")
    reader.Update()
    volume_state_1 = to_volume_state(reader.GetOutput())

    humerus_1 = dash_vtk.View([
        dash_vtk.VolumeRepresentation([
            dash_vtk.VolumeController(),
            dash_vtk.Volume(state=volume_state_1),
        ]),
    ])
    
    if use_flipped:
        if show_flip and os.path.exists(f"vtk_images/humerus_{ct_2}_segm_flip_mirrored.vtk"):
            reader.SetFileName(f"vtk_images/humerus_{ct_2}_segm_flip_mirrored.vtk")
        else:
            reader.SetFileName(f"vtk_images/humerus_{ct_2}_segm_flip.vtk")
    else:
        reader.SetFileName(f"vtk_images/humerus_{ct_2}_segm.vtk")
    reader.Update()
    volume_state_2 = to_volume_state(reader.GetOutput())

    humerus_2 = dash_vtk.View([
        dash_vtk.VolumeRepresentation([
            dash_vtk.VolumeController(),
            dash_vtk.Volume(state=volume_state_2),
        ]),
    ])
    
    return humerus_0,humerus_1,humerus_2,ct_0,ct_1,ct_2

In [ ]:
# Con 2 callback diversi riesco a separare i due caricamenti e aggiornare solo la riga che cambia
use_flip = True
show_flip=True

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

dropdown_style = {
    "width":"50%",
    "margin-top":"1em",
    "margin-bottom":"1em",
}
dropdown_list = [f"Cluster {x}" for x in range(len(indexes_ct_flip))] if use_flip else [f"Cluster {x}" for x in range(len(indexes_ct))]

app.layout = dbc.Container([
    dcc.Dropdown(dropdown_list,"Cluster 0",id="dropdown_1",style=dropdown_style,clearable=False),
    dbc.Row(id="row_1"),
    dcc.Dropdown(dropdown_list,"Cluster 1",id="dropdown_2",style=dropdown_style,clearable=False),
    dbc.Row(id="row_2",style={"margin-bottom":"5em"}),
])

@app.callback(
    Output("row_1","children"),
    Input("dropdown_1","value"),
)
def update_row_1(drop_1):  
    cluster_1 = int(drop_1.split()[1])
    
    humerus_1_1,humerus_1_2,humerus_1_3,ct_1_1,ct_1_2,ct_1_3 = create_row(cluster_1,use_flipped=use_flip,show_flip=show_flip)
    row_1 = [
        create_col(f"Cluster {cluster_1} Omero {ct_1_1}",humerus_1_1),
        create_col(f"Cluster {cluster_1} Omero {ct_1_2}",humerus_1_2),
        create_col(f"Cluster {cluster_1} Omero {ct_1_3}",humerus_1_3),
    ]
    
    return row_1

@app.callback(
    Output("row_2","children"),
    Input("dropdown_2","value"),
)
def update_row_2(drop_2):  
    cluster_2 = int(drop_2.split()[1])

    humerus_2_1,humerus_2_2,humerus_2_3,ct_2_1,ct_2_2,ct_2_3 = create_row(cluster_2,use_flipped=use_flip,show_flip=show_flip)
    row_2 = [
        create_col(f"Cluster {cluster_2} Omero {ct_2_1}",humerus_2_1),
        create_col(f"Cluster {cluster_2} Omero {ct_2_2}",humerus_2_2),
        create_col(f"Cluster {cluster_2} Omero {ct_2_3}",humerus_2_3),
    ]
    
    return row_2

app.title = "Humerus 3D Rendering"
app.run(jupyter_mode="external",host="0.0.0.0",port="2323")

In [ ]:
indexes_ct,indexes_ct_flip